In [4]:
import pandas as pd
import numpy as np
import pickle
import os
from pathlib import Path
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

In [5]:
# Load the dataset


# Define the base directory containing data files
base_dir = "/Users/mjh/code/Obispodino/cvino/raw_data/last"  # Updated to absolute path # Updated to absolute path
wines_file = os.path.join(base_dir, "XWines_Full_100K_wines.csv")
wines_data = pd.read_csv(wines_file)
wd = wines_data
wd.head()

,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201..."
1,100002,Ancellotta,Red,Varietal/100%,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2016, 2015, 2014, 2013, 2012, 2011, 2010, 200..."
2,100003,Cabernet Sauvignon,Red,Varietal/100%,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,BR,Brazil,1001,Serra Gaúcha,10002,Castellamare,https://www.emporiocastellamare.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
3,100004,Virtus Moscato,White,Varietal/100%,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10003,Monte Paschoal,http://www.montepaschoal.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."
4,100005,Maison de Ville Cabernet-Merlot,Red,Assemblage/Bordeaux Red Blend,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,BR,Brazil,1001,Serra Gaúcha,10000,Aurora,http://www.vinicolaaurora.com.br,"[2021, 2020, 2019, 2018, 2017, 2016, 2015, 201..."


In [6]:
# def geocode_regions(df, region_column='RegionName', cache_file='geocoding_cache.pkl', min_delay=1.0):
#     """
#     Geocode region names with caching and rate limiting

#     Parameters:
#         df (pd.DataFrame): Input DataFrame
#         region_column (str): Name of region column
#         cache_file (str): Path to cache file
#         min_delay (float): Minimum delay between API requests (seconds)

#     Returns:
#         pd.DataFrame: DataFrame with latitude/longitude columns
#     """
#     # Initialize geocoder with rate limiter
#     geolocator = Nominatim(user_agent="regional_analysis_app")
#     geocode = RateLimiter(geolocator.geocode, min_delay_seconds=min_delay)

#     # Load existing cache
#     cache_path = Path(cache_file)
#     if cache_path.exists():
#         with open(cache_path, 'rb') as f:
#             geocode_cache = pickle.load(f)
#     else:
#         geocode_cache = {}

#     # Get unique regions needing geocoding
#     unique_regions = df[region_column].dropna().unique()
#     new_regions = [r for r in unique_regions if r not in geocode_cache]

#     print(f"Found {len(unique_regions)} unique regions ({len(new_regions)} new)")

#     # Geocode new regions with retry logic
#     for region in new_regions:
#         for attempt in range(3):  # 3 retry attempts
#             try:
#                 location = geocode(region)
#                 geocode_cache[region] = {
#                     'latitude': location.latitude if location else np.nan,
#                     'longitude': location.longitude if location else np.nan
#                 }
#                 break
#             except Exception as e:
#                 if attempt == 2:  # Final attempt failed
#                     geocode_cache[region] = {'latitude': np.nan, 'longitude': np.nan}
#                     print(f"Failed to geocode {region} after 3 attempts")
#                 continue

#     # Save updated cache
#     with open(cache_path, 'wb') as f:
#         pickle.dump(geocode_cache, f)

#     # Create coordinate columns
#     df['latitude'] = df[region_column].map(lambda x: geocode_cache.get(x, {}).get('latitude'))
#     df['longitude'] = df[region_column].map(lambda x: geocode_cache.get(x, {}).get('longitude'))

#     return df #.drop(columns=[region_column])


In [7]:
# # Usage
# wd = geocode_regions(wd, min_delay=2.0)
# print("Geocoding completed.")

# # Identify rows where geocoding failed (latitude is null) after geocoding
# not_found_df = wd[wd['latitude'].isnull()]

# # Get a unique list of the region names that were not found
# unresolved_regions = not_found_df['RegionName'].unique().tolist()

# if unresolved_regions:
#     print("The following regions could not be geocoded:")
#     for region in unresolved_regions:
#         print(f"- {region}")
# else:
#     print("All regions were successfully geocoded.")

In [8]:
def geocode_regions_online(RegionName, region_column='RegionName', min_delay=2.0): #, cache_file='geocoding_cache.pkl'
    """
    Geocode region names with caching and rate limiting

    Parameters:
        df (pd.DataFrame): Input DataFrame
        region_column (str): Name of region column
        cache_file (str): Path to cache file
        min_delay (float): Minimum delay between API requests (seconds)

    Returns:
        pd.DataFrame: DataFrame with latitude/longitude columns
    """
    # Initialize geocoder with rate limiter
    geolocator = Nominatim(user_agent="regional_analysis_app")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=min_delay)

    # Load existing cache
    # cache_path = Path(cache_file)
    # if cache_path.exists():
    #     with open(cache_path, 'rb') as f:
    #         geocode_cache = pickle.load(f)
    # else:
    geocode_cache = {}

    # Get unique regions needing geocoding
    region = RegionName
    # new_regions = [r for r in unique_regions if r not in geocode_cache]

    # Geocode new regions with retry logic

    for attempt in range(3):  # 3 retry attempts
        try:
            location = geocode(region)
            geocode_cache[region] = {
                'latitude': location.latitude if location else np.nan,
                'longitude': location.longitude if location else np.nan
            }
            break
        except Exception as e:
            if attempt == 2:  # Final attempt failed
                geocode_cache[region] = {'latitude': np.nan, 'longitude': np.nan}
                print(f"Failed to geocode {region} after 3 attempts")
            continue
    return geocode_cache


In [9]:
unpickled_df = pd.read_pickle("./geocoding_cache.pkl")
unpickled_df

{'Serra Gaúcha': {'latitude': -29.1631418, 'longitude': -51.4955259},
 'Campanha Gaúcha': {'latitude': -23.5371755, 'longitude': -46.4168075},
 'Alentejano': {'latitude': -22.9056774, 'longitude': -43.1763427},
 'Santiago': {'latitude': 9.8694792, 'longitude': -83.7980749},
 'Vinho Verde': {'latitude': 41.0674998, 'longitude': -8.2372694},
 'Porto': {'latitude': 41.1494512, 'longitude': -8.6107884},
 'Rio Grande do Sul': {'latitude': -29.8425284, 'longitude': -53.7680577},
 'Vale dos Vinhedos': {'latitude': -29.1790429, 'longitude': -51.5789598},
 'São Paulo': {'latitude': -23.5506507, 'longitude': -46.6333824},
 'Serra Negra': {'latitude': -22.6126459, 'longitude': -46.699986},
 'Santa Catarina': {'latitude': -27.0628367, 'longitude': -51.114965},
 'Vale do São Francisco': {'latitude': -9.4115177, 'longitude': -40.5147187},
 'Campos de Cima da Serra': {'latitude': -28.4299489, 'longitude': -50.897498},
 'Minas Gerais': {'latitude': -18.5264844, 'longitude': -44.1588654},
 'Pinto Bande

In [10]:
wd_new = wd[['RegionName','Country']].drop_duplicates()
wd_new

,RegionName,Country
0,Serra Gaúcha,Brazil
8,Campanha Gaúcha,Brazil
9,Alentejano,Portugal
10,Santiago,Chile
11,Vinho Verde,Portugal
...,...,...
100524,Dodecanese,Greece
100527,Chania,Greece
100533,Ohrid,North Macedonia
100610,Sharon,Israel


In [11]:
my_dict = pd.Series(wd_new['Country'].values,index=wd_new['RegionName']).to_dict()
print(my_dict)

{'Serra Gaúcha': 'Brazil', 'Campanha Gaúcha': 'Brazil', 'Alentejano': 'Portugal', 'Santiago': 'Chile', 'Vinho Verde': 'Portugal', 'Porto': 'Portugal', 'Rio Grande do Sul': 'Brazil', 'Vale dos Vinhedos': 'Brazil', 'São Paulo': 'Brazil', 'Serra Negra': 'Brazil', 'Santa Catarina': 'Brazil', 'Vale do São Francisco': 'Brazil', 'Campos de Cima da Serra': 'Brazil', 'Minas Gerais': 'Brazil', 'Pinto Bandeira': 'Brazil', 'Planalto Catarinense': 'Brazil', 'Maipo Valley': 'Chile', 'Canelones': 'Uruguay', 'Colchagua Valley': 'Chile', 'Toscana': 'Italy', 'Maipu': 'Argentina', 'Serra do Sudeste': 'Brazil', 'Mendoza': 'Argentina', 'Paraná': 'Brazil', 'Maule Valley': 'Chile', 'California': 'United States', 'Champagne Premier Cru': 'France', 'Amarone della Valpolicella Classico': 'Italy', 'Goiás': 'Brazil', 'South Eastern Australia': 'Australia', 'Stara Zagora': 'Bulgaria', 'Alentejo': 'Portugal', 'Douro': 'Portugal', 'Evora': 'Portugal', 'Lisboa': 'Portugal', 'Península de Setúbal': 'Portugal', 'Monção

In [12]:
def geocode_regions2(df, region_column='RegionName',country_column = 'Country', cache_file='geocoding_cache.pkl', min_delay=1.0):
    """
    Geocode region names with caching and rate limiting

    Parameters:
        df (pd.DataFrame): Input DataFrame
        region_column (str): Name of region column
        cache_file (str): Path to cache file
        min_delay (float): Minimum delay between API requests (seconds)

    Returns:
        pd.DataFrame: DataFrame with latitude/longitude columns
    """
    # Initialize geocoder with rate limiter
    geolocator = Nominatim(user_agent="regional_analysis_app")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=min_delay)

    # create coutry vs city dictionary
    df_dict_df = df[[region_column,country_column]].drop_duplicates()
    df_dict = pd.Series(df_dict_df[country_column].values,index=df_dict_df[region_column]).to_dict()

    # Load existing cache
    cache_path = Path(cache_file)
    if cache_path.exists():
        with open(cache_path, 'rb') as f:
            geocode_cache = pickle.load(f)
    else:
        geocode_cache = {}

    # Get unique regions needing geocoding
    unique_regions = df[region_column].dropna().unique()

    new_regions = [r for r in unique_regions if r not in geocode_cache]

    print(f"Found {len(unique_regions)} unique regions ({len(new_regions)} new)")

    # Geocode new regions with retry logic
    for region in new_regions:
        for attempt in range(3):  # 3 retry attempts
            try:
                location = geocode(region)
                geocode_cache[region] = {
                    'latitude': location.latitude if location else np.nan,
                    'longitude': location.longitude if location else np.nan
                }

                if location.latitude == 'none':
                    location = geocode(df_dict[region])
                    geocode_cache[region] = {
                    'latitude': location.latitude if location else np.nan,
                    'longitude': location.longitude if location else np.nan}

                break
            except Exception as e:
                if attempt == 2:  # Final attempt failed
                    geocode_cache[region] = {'latitude': np.nan, 'longitude': np.nan}
                    print(f"Failed to geocode {region} after 3 attempts")
                continue


    # Save updated cache
    with open(cache_path, 'wb') as f:
        pickle.dump(geocode_cache, f)

    # Create coordinate columns
    df['latitude'] = df[region_column].map(lambda x: geocode_cache.get(x, {}).get('latitude'))
    df['longitude'] = df[region_column].map(lambda x: geocode_cache.get(x, {}).get('longitude'))

    return df #.drop(columns=[region_column])


In [13]:
# Usage
wd = geocode_regions2(wd, min_delay=2.0)
print("Geocoding completed.")

# Identify rows where geocoding failed (latitude is null) after geocoding
not_found_df = wd[wd['latitude'].isnull()]

# Get a unique list of the region names that were not found
unresolved_regions = not_found_df['RegionName'].unique().tolist()

if unresolved_regions:
    print("The following regions could not be geocoded:")
    for region in unresolved_regions:
        print(f"- {region}")
else:
    print("All regions were successfully geocoded.")

Found 2160 unique regions (0 new)
Geocoding completed.
All regions were successfully geocoded.


In [14]:
geocode_regions_online('Chile')

{'Chile': {'latitude': -31.7613365, 'longitude': -71.3187697}}

In [15]:
location = geocode_regions_online('Valley Chile')
location

{'Valley Chile': {'latitude': 35.9881683, 'longitude': -114.9814307}}

In [16]:
geolocator = Nominatim(user_agent="my_app")
location = geolocator.reverse("35.9881683, -114.9814307")
location

Location(Chile Valley Avenue, Mission Hills, Southeast Henderson, Las Vegas Valley, Clark County, Nevada, 89015, United States, (35.9881683, -114.9814307, 0.0))

In [17]:
# WineID is a label, lets drop it and some more...
wd = wd.drop(columns=['WineID', 'Elaborate', 'Code', 'Country', 'RegionID', 'WineryID', 'Website', 'Vintages', 'WineryName'])
wd.head()

,WineName,Type,Grapes,Harmonize,ABV,Body,Acidity,RegionName,latitude,longitude
0,Espumante Moscatel,Sparkling,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,Serra Gaúcha,-29.163142,-51.495526
1,Ancellotta,Red,['Ancellotta'],"['Beef', 'Barbecue', 'Codfish', 'Pasta', 'Pizz...",12.0,Medium-bodied,Medium,Serra Gaúcha,-29.163142,-51.495526
2,Cabernet Sauvignon,Red,['Cabernet Sauvignon'],"['Beef', 'Lamb', 'Poultry']",12.0,Full-bodied,High,Serra Gaúcha,-29.163142,-51.495526
3,Virtus Moscato,White,['Muscat/Moscato'],['Sweet Dessert'],12.0,Medium-bodied,Medium,Serra Gaúcha,-29.163142,-51.495526
4,Maison de Ville Cabernet-Merlot,Red,"['Cabernet Sauvignon', 'Merlot']","['Beef', 'Lamb', 'Game Meat', 'Poultry']",11.0,Full-bodied,Medium,Serra Gaúcha,-29.163142,-51.495526


In [18]:
# DATA CLEANING STAGE - Step 4: Prepare Features for k-NN Model
print("=== PREPARING FEATURES FOR k-NN MODEL ===")

=== PREPARING FEATURES FOR k-NN MODEL ===


In [20]:
wd [['Type']]

,Type
0,Sparkling
1,Red
2,Red
3,White
4,Red
...,...
100641,White
100642,Dessert
100643,White
100644,White


In [19]:
type_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
type_encoded = type_encoder.fit_transform(wd [['Type']])
type_encoder

,categories,'auto'
,drop,None
,sparse_output,False
,dtype,<class 'numpy.float64'>
,handle_unknown,'ignore'
,min_frequency,None
,max_categories,None
,feature_name_combiner,'concat'


In [21]:
type_encoder.categories_[0]

array(['Dessert', 'Dessert/Port', 'Red', 'Rosé', 'Sparkling', 'White'],
      dtype=object)

In [36]:
import os
import pandas as pd
from pathlib import Path

LOCAL_DATA_PATH = os.path.join(os.path.expanduser('~'), "code", "Obispodino", "cvino", "raw_data")
clean_wine_path = os.path.join(LOCAL_DATA_PATH, "wines_clean.csv")
clean_ratings_path = os.path.join(LOCAL_DATA_PATH, "ratings_clean.csv")

if Path(clean_wine_path).is_file() and Path(clean_ratings_path).is_file():
    print( "\nLoad clean wine data from local CSV..." )
    wines_clean_df = pd.read_csv(clean_wine_path)
    wines_clean_df['Grapes_list'] =  wines_clean_df['Grapes'].apply(lambda x: eval(x) if isinstance(x, str) else x)
    print( "\nLoad clean rating data from local CSV..." )
    ratings_clean_df = pd.read_csv(clean_ratings_path)



Load clean wine data from local CSV...

Load clean rating data from local CSV...


/var/folders/4h/7hhtxprx1fn6ljyqcdhz50q00000gn/T/ipykernel_3337/4271167936.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  ratings_clean_df = pd.read_csv(clean_ratings_path)


In [29]:
wine_df = wines_clean_df.merge(ratings_clean_df, on='WineID', how='left')

In [ ]:
wine_df.shape


,WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,...,RegionID,RegionName,WineryID,WineryName,Website,Vintages,latitude,longitude,Grapes_list,Rating
0,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,...,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",-29.163142,-51.495526,[Muscat/Moscato],5.0
1,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,...,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",-29.163142,-51.495526,[Muscat/Moscato],3.0
2,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,...,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",-29.163142,-51.495526,[Muscat/Moscato],4.0
3,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,...,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",-29.163142,-51.495526,[Muscat/Moscato],5.0
4,100001,Espumante Moscatel,Sparkling,Varietal/100%,['Muscat/Moscato'],"['Pork', 'Rich Fish', 'Shellfish']",7.5,Medium-bodied,High,BR,...,1001,Serra Gaúcha,10001,Casa Perini,http://www.vinicolaperini.com.br,"[2020, 2019, 2018, 2017, 2016, 2015, 2014, 201...",-29.163142,-51.495526,[Muscat/Moscato],2.5


In [31]:
len(wine_df.columns), wine_df.columns

(21,
 Index(['WineID', 'WineName', 'Type', 'Elaborate', 'Grapes', 'Harmonize', 'ABV',
        'Body', 'Acidity', 'Code', 'Country', 'RegionID', 'RegionName',
        'WineryID', 'WineryName', 'Website', 'Vintages', 'latitude',
        'longitude', 'Grapes_list', 'Rating'],
       dtype='object'))

In [ ]:
import os
preprocessor_file = os.path.join(os.path.expanduser('~'), "code", "Obispodino", "cvino", 'models',"preprocessor.pkl")
with open(preprocessor_file, 'rb') as f:
    preprocessor = pickle.load(f)

In [68]:
imputer = preprocessor.named_transformers_["num"].named_steps["imputer"]
print(imputer.statistics_)

[13.4       42.965916   2.8390454  3.79      58.         0.64     ]


In [97]:
wine_type='Red'        # Wine type (Red, White, Rosé, Sparkling, etc.)
grape_varieties='Syrah/Shiraz'      # Grape varieties (single string or list)
body='Full-bodied'           # Body ['Very light-bodied', 'Light-bodied', 'Medium-bodied', 'Full-bodied', 'Very full-bodied']
acidity= 'Low'           # Acidity (Low, Medium, High)
abv=None                  # Alcohol by volume percentage
country='Australia'               # Country of origin
region_name='Maule Valley'  # Region name
n_recommendations=5

In [98]:

latitude_in = 42.965916,
longitude_in = 2.8390454,


    # create a DataFrame
X_pred = pd.DataFrame(dict(
Type =wine_type,
ABV=abv,
Body=body,
Acidity=acidity,
Country=country,
RegionName=region_name,
latitude = latitude_in,
longitude = longitude_in,
Grapes_list = grape_varieties,
avg_rating = [3.79],
rating_count = [0],
rating_std = [0]
))

In [99]:
X_pred

,Type,ABV,Body,Acidity,Country,RegionName,latitude,longitude,Grapes_list,avg_rating,rating_count,rating_std
0,Red,None,Full-bodied,Low,Australia,Maule Valley,42.965916,2.839045,Syrah/Shiraz,3.79,0,0


In [100]:
# Ensure latitude and longitude are scalar floats, not lists
# X_pred['latitude'] = X_pred['latitude'].apply(lambda x: x[0] if isinstance(x, (list, tuple)) else x)
# X_pred['longitude'] = X_pred['longitude'].apply(lambda x: x[0] if isinstance(x, (list, tuple)) else x)
X_pred_cleaned = X_pred.replace({None: np.nan})
X_processed = preprocessor.transform(X_pred_cleaned)

/var/folders/4h/7hhtxprx1fn6ljyqcdhz50q00000gn/T/ipykernel_3337/1342353716.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_pred_cleaned = X_pred.replace({None: np.nan})


In [101]:
column_names = get_feature_names_out(preprocessor)

In [102]:
X_df = pd.DataFrame(X_processed, columns=column_names, index=X_pred.index)
X_df

,Type_Dessert,Type_Dessert/Port,Type_Red,Type_Rosé,Type_Sparkling,Type_White,Grape_Cabernet Sauvignon,Body,Acidity,ABV,latitude,longitude,avg_rating,rating_count,rating_std
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.268,0.802608,0.485189,0.677333,-0.000182,0.0


In [80]:
def get_feature_names_out(ct):
    names = []
    for name, trans, cols in ct.transformers_:
        if hasattr(trans, 'get_feature_names_out'):
            names.extend(trans.get_feature_names_out())
        else:
            names.extend(cols)
    return names